In [20]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error

#### Question 1: Read the `yellow_tripdata_2023-01.parquet` dataset and get the number of columns,

In [3]:
df = pd.read_parquet("dataset/yellow_tripdata_2023-01.parquet")

In [6]:
print(df.columns)
print(len(df.columns))

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')
19


#### Question 2: Computing duration
- To compute the duration of a trip, I take two columns in the dataframe, which are `tpep_pickup_datetime` and `tpep_dropoff_datetime`, then let's the `dropoff time` subtract by the `pickup time`
- Then, convert the duration type to minute
- Finally, calcalate the standard deviation of the durtion by applying the `std()` function.

In [7]:
df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
# Convert timedelta to minute
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [8]:
print(df.duration)

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3066766, dtype: float64


In [9]:
# Calculating the standard deviation of the duration
print(df.duration.std())

42.59435124195458


#### Question 3: Dropping outliers
- In this question, I first get the number of rows before drop the outliers.
- Dropping the outlier by keeping the duration between 1 and 60.
- To calculate the rate, I divide the number of rows after dropout by rows before dropout.

In [12]:
rows_before = df.shape[0]

df = df[(df.duration >= 1) & (df.duration <= 60)]

rows_after = df.shape[0]

print(f"{rows_before=}")
print(f"{rows_after=}")
print(f"Ratio = {rows_after / rows_before}")

rows_before=3066766
rows_after=3009173
Ratio = 0.9812202822125979


#### Question 4: One-hot encoding

In [13]:
features = ["PULocationID", "DOLocationID"]
df[features] = df[features].astype(str)

In [17]:
train_dicts = df[features].to_dict(orient="records")

dict_vectorizer = DictVectorizer()

X_train = dict_vectorizer.fit_transform(train_dicts)
y_train = df["duration"].values

In [19]:
print(X_train.shape[1])

515


#### Question 5: Training a model

In [18]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [22]:
y_pred = model.predict(X_train)

In [23]:
print(f"RMSE: {root_mean_squared_error(y_train, y_pred)}")

RMSE: 7.649261929201487


#### Question 6: Evaluting the model

In [24]:
df_val = pd.read_parquet("dataset/yellow_tripdata_2023-02.parquet")

In [25]:
df_val.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
count,2.913955e+06,2913955,2913955,2.837138e+06,2.913955e+06,2.837138e+06,2.913955e+06,2.913955e+06,2.913955e+06,2.913955e+06,2.913955e+06,2.913955e+06,2.913955e+06,2.913955e+06,2.913955e+06,2.913955e+06,2.837138e+06,2.837138e+06
mean,1.731282e+00,2023-02-15 00:38:01.639923,2023-02-15 00:54:02.594499,1.352847e+00,3.868058e+00,1.514554e+00,1.662570e+02,1.642317e+02,1.182896e+00,1.822038e+01,1.560472e+00,4.878752e-01,3.384825e+00,5.113983e-01,9.813052e-01,2.689848e+01,2.285299e+00,9.677754e-02
min,1.000000e+00,2008-12-31 23:05:06,2008-12-31 23:31:24,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-7.540500e+02,-7.500000e+00,-5.000000e-01,-9.300000e+01,-5.700000e+01,-1.000000e+00,-7.575500e+02,-2.500000e+00,-1.250000e+00
25%,1.000000e+00,2023-02-08 08:43:13,2023-02-08 08:59:34,1.000000e+00,1.060000e+00,1.000000e+00,1.320000e+02,1.130000e+02,1.000000e+00,8.600000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,1.000000e+00,1.548000e+01,2.500000e+00,0.000000e+00
50%,2.000000e+00,2023-02-14 22:33:21,2023-02-14 22:48:15,1.000000e+00,1.790000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.280000e+01,1.000000e+00,5.000000e-01,2.800000e+00,0.000000e+00,1.000000e+00,2.020000e+01,2.500000e+00,0.000000e+00
75%,2.000000e+00,2023-02-22 08:34:18,2023-02-22 08:50:06,1.000000e+00,3.300000e+00,1.000000e+00,2.340000e+02,2.340000e+02,1.000000e+00,2.030000e+01,2.500000e+00,5.000000e-01,4.200000e+00,0.000000e+00,1.000000e+00,2.870000e+01,2.500000e+00,0.000000e+00
max,6.000000e+00,2023-03-07 13:01:28,2023-03-07 13:11:17,9.000000e+00,3.350043e+05,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,2.203100e+03,2.080000e+01,4.000000e+00,4.829000e+02,1.099000e+02,1.000000e+00,2.208100e+03,2.750000e+00,1.250000e+00
std,4.524952e-01,NaN,NaN,8.891265e-01,2.684273e+02,6.633395e+00,6.413230e+01,6.975406e+01,5.297219e-01,1.749819e+01,1.792708e+00,1.006192e-01,3.838069e+00,2.024488e+00,1.873103e-01,2.184110e+01,7.609081e-01,3.388268e-01


In [26]:
print(df_val.columns)
print(len(df_val.columns))

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')
19


In [27]:
df_val["duration"] = df_val["tpep_dropoff_datetime"] - df_val["tpep_pickup_datetime"]
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

In [28]:
df_val.duration

0           1.683333
1           0.233333
2           0.233333
3          32.083333
4          13.300000
             ...    
2913950    19.000000
2913951    11.133333
2913952    14.000000
2913953     7.000000
2913954     9.800000
Name: duration, Length: 2913955, dtype: float64

In [29]:
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

In [30]:
df_val[features] = df_val[features].astype(str)

In [31]:
val_dicts = df_val[features].to_dict(orient="records")

X_val = dict_vectorizer.transform(val_dicts)
y_val = df_val["duration"].values

In [32]:
y_val_pred = model.predict(X_val)

In [33]:
print(f"RMSE val: {root_mean_squared_error(y_val, y_val_pred)}")

RMSE val: 7.811819793542861
